## Reading MIDI Files

In [1]:
import midi

The authors of these files structured them in at least two distinct ways. 

### Single-track files

In [2]:
promises = midi.read_midifile('../midi/pop/10000_Promises.mid')
print len(promises)

1


In our first song, 10000 Promises, we see a file with a **single track** containing:

#### KeySignatureEvent -- optional melodic key for the song (think number of sharps and flats)

In [3]:
promises[0][0]

midi.KeySignatureEvent(tick=0, data=[0, 0])

#### SmpteOffsetEvent -- time code specification (not important for now)

In [4]:
promises[0][1]

midi.SmpteOffsetEvent(tick=0, data=[96, 0, 0, 0, 0])

#### SetTempoEvent -- the number of microseconds to assign to one beat (base 256 representation)

In [5]:
promises[0][2]

midi.SetTempoEvent(tick=0, data=[6, 162, 94])

          Note: if the microseconds per beat are 

$$6 \cdot 256^{2} + 162 \cdot 256 + 94 = 393216 + 41472 + 94 = 434782, $$
          
          then the beats per minute are 
$$60000000 \div 434782 = 138.$$

We can use the following methods to easily transform units:

In [6]:
promises[0][2].get_mpqn()

434782

In [7]:
promises[0][2].get_bpm()

138.00019320027047

#### ProgramChangeEvent -- IMPORTANT -- change the instrument

In [8]:
promises[0][4]

midi.ProgramChangeEvent(tick=0, channel=0, data=[1])

Format: at time `tick`, assign instrument number `data` to `channel`, using the instrument numbers in the chart below.

For example, the event above says to assign a Bright Acoustic Piano (the second instrument) to Channel 1 at time 0. Note the annoying differences in computer-counting versus musician-counting here.

![General MIDI Level 1 Instrument Patch Map](../pictures/GM1Patch.png)
![General MIDI Level 1 Instrument Patch Map](../pictures/GM1Patch2.png)
        
        Graphics copyright © 1995-2015 MIDI Manufacturers Association Incorporated.

#### ControlChangeEvent -- change some global feature of the instrument on `channel`

In [9]:
promises[0][5]

midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127])

Format: at time `tick`, set control number `data[0]` to control value `data[1]` on `channel`.

For example, the event above assigns maximum volume to Channel 1 at time 0.

**Some other control number examples:**
- 1 - Modulation Wheel - values: 0 - 127
- 7 - Volume
- 10 - Pan
- 64 - Sustain pedal

Note: control numbers 0 (MSB) and 32 (LSB) are used to switch banks (i.e. more sets of the same instruments or other instruments altogether). We will be mainly working with only one bank, so we can ignore these for now.

#### NoteOnEvent -- play a note

In [10]:
promises[0][38]

midi.NoteOnEvent(tick=117, channel=0, data=[73, 53])

Format: at time `tick`, play note `data[0]` at velocity `data[1]`.

For example, the event above plays the note C#5 (about an octave above middle C#) with "velocity" 53 (out of 127) exactly 117 ms after the last event.

Note: there are also **NoteOffEvent**s, to explicitly end the notes; these seem optional except in circumstances where there is no natural sonic decay (as there is for say, a guitar string) in the chosen instrument.

#### EndOfTrackEvent -- just what it sounds like

In [11]:
promises[0][-1]

midi.EndOfTrackEvent(tick=0, data=[])

**Note well:** 

You may have noticed that the tick values are *in relation to the last event*, and also that events may be simultaneous. We can use the following two methods to change the `tick` values from relative to absolute time.

In [12]:
promises.make_ticks_abs()
promises[0][-1]

midi.EndOfTrackEvent(tick=63402, data=[])

In [13]:
promises.make_ticks_rel()
promises[0][-1]

midi.EndOfTrackEvent(tick=0, data=[])

All of the channels are controlled from a single track in this setup, as opposed to the following structure.

### Multi-track files

In [14]:
oceans = midi.read_midifile('../midi/pop/1000_Oceans.mid')
print len(oceans)

23


This file, 1000 Oceans, contains multiple tracks ...

In [15]:
for i in range(len(oceans)):
    assert isinstance(oceans[i], midi.containers.Track)

... some providing tempo-related information ...

In [16]:
oceans[0].make_ticks_abs()
oceans[0]

midi.Track(\
  [midi.TimeSignatureEvent(tick=0, data=[4, 2, 96, 8]),
   midi.SequencerSpecificEvent(tick=0, data=[0, 0, 65]),
   midi.SetTempoEvent(tick=0, data=[13, 98, 136]),
   midi.SetTempoEvent(tick=105984, data=[13, 98, 136]),
   midi.SetTempoEvent(tick=106136, data=[13, 128, 219]),
   midi.SetTempoEvent(tick=106288, data=[13, 159, 185]),
   midi.SetTempoEvent(tick=106440, data=[13, 191, 36]),
   midi.SetTempoEvent(tick=106592, data=[13, 217, 195]),
   midi.SetTempoEvent(tick=106744, data=[13, 250, 62]),
   midi.SetTempoEvent(tick=106896, data=[14, 27, 83]),
   midi.SetTempoEvent(tick=107048, data=[14, 55, 93]),
   midi.SetTempoEvent(tick=107200, data=[14, 89, 150]),
   midi.SetTempoEvent(tick=107352, data=[14, 124, 119]),
   midi.SetTempoEvent(tick=107504, data=[14, 154, 9]),
   midi.EndOfTrackEvent(tick=107504, data=[])])

... and others providing information about instruments and notes:

In [17]:
oceans[1][:10]

midi.Track(\
  [midi.TrackNameEvent(tick=0, text='"1000 Oceans" by Tori Amos', data=[34, 49, 48, 48, 48, 32, 79, 99, 101, 97, 110, 115, 34, 32, 98, 121, 32, 84, 111, 114, 105, 32, 65, 109, 111, 115]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[10, 63]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[91, 74]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[93, 7]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[65, 103]),
   midi.NoteOnEvent(tick=0, channel=0, data=[58, 103]),
   midi.NoteOffEvent(tick=572, channel=0, data=[65, 65])])

In [18]:
oceans[2][:10]

midi.Track(\
  [midi.TrackNameEvent(tick=0, text='made MIDI by Chris Howe', data=[109, 97, 100, 101, 32, 77, 73, 68, 73, 32, 98, 121, 32, 67, 104, 114, 105, 115, 32, 72, 111, 119, 101]),
   midi.ControlChangeEvent(tick=0, channel=1, data=[0, 0]),
   midi.ProgramChangeEvent(tick=0, channel=1, data=[0]),
   midi.ControlChangeEvent(tick=0, channel=1, data=[10, 63]),
   midi.ControlChangeEvent(tick=0, channel=1, data=[91, 78]),
   midi.ControlChangeEvent(tick=0, channel=1, data=[93, 0]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.NoteOnEvent(tick=0, channel=1, data=[46, 100]),
   midi.NoteOnEvent(tick=0, channel=1, data=[53, 100]),
   midi.NoteOffEvent(tick=572, channel=1, data=[46, 64])])

Here, a typical usage is to assign one track to each channel. The only exception in our sample file occurs in track 14, which seems to be pretty useless to begin with.

In [19]:
for i in range(len(oceans)):
    channel = None
    for event in oceans[i]:
        if 'channel' not in dir(event):
            continue
        elif channel == None:
            channel = event.channel
        else:
            if channel != event.channel:
                print i, event

14 midi.NoteOnEvent(tick=0, channel=1, data=[46, 87])
14 midi.NoteOnEvent(tick=0, channel=1, data=[53, 87])
14 midi.NoteOnEvent(tick=0, channel=2, data=[46, 100])
14 midi.NoteOnEvent(tick=0, channel=3, data=[34, 59])
14 midi.NoteOnEvent(tick=0, channel=4, data=[46, 115])
14 midi.NoteOnEvent(tick=0, channel=9, data=[41, 46])
14 midi.NoteOnEvent(tick=0, channel=9, data=[46, 27])
14 midi.NoteOffEvent(tick=0, channel=1, data=[46, 0])
14 midi.NoteOffEvent(tick=0, channel=1, data=[53, 0])
14 midi.NoteOffEvent(tick=0, channel=2, data=[46, 0])
14 midi.NoteOffEvent(tick=0, channel=3, data=[34, 0])
14 midi.NoteOffEvent(tick=0, channel=4, data=[46, 0])
14 midi.NoteOffEvent(tick=0, channel=9, data=[41, 0])
14 midi.NoteOffEvent(tick=124, channel=9, data=[46, 64])
14 midi.NoteOnEvent(tick=4, channel=9, data=[46, 27])
14 midi.NoteOffEvent(tick=256, channel=9, data=[46, 0])


Indeed, sometimes a number of tracks are of no interest to us.

In [20]:
oceans[11]

midi.Track(\
  [midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.EndOfTrackEvent(tick=0, data=[])])

Our overall plan will be to check for meta-events like instrument assignment and tempo settings, then to store the relevant note events from each channel to create a time series for each instrument's part.